# Diet Problem written in Julia Language

## 1. First of all, you need to upload certain packages

In [ ]:
Pkg.add("DataDeps")
Pkg.add("ExcelFiles")
Pkg.add("DataFrames")


In [ ]:
using JuMP
using GLPKMathProgInterface
using ExcelFiles, DataFrames

## 2. Load your database by using DataFrame

In [ ]:
const nutrients = DataFrame(load("ausnut.xls", "Food Nutrient Database"))

## 3. Create functions to solve Diet Problem

In [ ]:
using JuMP
using LinearAlgebra
function new_diet(solver = GLPKSolverLP())
    m = Model(solver= solver)
    @variable(m, diet[1:size(nutrients,1)], lowerbound=0)
    m, diet 
end

### Function below takes certain rows and columns and read their values. Names of columns and rows depend on database you use

In [ ]:
function total_intake(nutrient_name, diet)
    nutrients[:, Symbol(nutrient_name)] ⋅ diet
end

function basic_nutrient_requirements(m::Model, diet, weight = 62) # average human is 62 kg
    intake(nutrient_name) = total_intake(nutrient_name, diet) 
    # ^ closure over the diet given as an argument
        
    @constraints(m, begin
        70 <= intake("Protein (g)") 
        70 <= intake("Total fat (g)") 
        24 <= intake("Total saturated fat (g)") 
        90 <= intake("Total sugars (g)") 
        intake("Alcohol (g)") <= 100
        310 <= intake("Available carbohydrates, with sugar alcohols (g)") 
        intake("Sodium (Na) (mg)") <= 2300 # Heart Council
        intake("Caffeine (mg)") <= 400
        25 <=intake("Dietary fibre (g)") 
        24 <= intake("Total trans fatty acids (mg)") 
        25 <= intake("Vitamin A retinol equivalents (µg)") <= 900
        1.2 <= intake("Thiamin (B1) (mg)") 
        400 <= intake("Dietary folate equivalents  (µg)") 
        1000 <= intake("Folic acid  (µg)") # https://ods.od.nih.gov/factsheets/Folate-HealthProfessional/
        75 <= intake("Vitamin C (mg)") <= 2000 ##https://en.wikipedia.org/wiki/Dietary_Reference_Intake
        95 <= intake("Iodine (I) (µg)") <=1100 ##https://en.wikipedia.org/wiki/Dietary_Reference_Intake
        6 <= intake("Iron (Fe) (mg)") <= 45 # #https://en.wikipedia.org/wiki/Dietary_Reference_Intake
        330 <= intake("Magnesium (Mg) (mg)") 
        580 <= intake("Phosphorus (P) (mg)") <= 4000 #https://en.wikipedia.org/wiki/Dietary_Reference_Intake
        4700 <= intake("Potassium (K) (mg)") 
        800 <= intake("Calcium (Ca) (mg)") <= 2500 #https://en.wikipedia.org/wiki/Dietary_Reference_Intake
        3.5*weight <= intake("Tryptophan (mg)") <= 0.5*weight*1600 #0.5 * LD50 https://books.google.com.au/books?id=he7LBQAAQBAJ&pg=PA210&lpg=PA210&dq=Tryptophan+LD-50&source=bl&ots=yzi5Hx7hM2&sig=sd78LGAnavIwtT5AP-L4fzsFLZM&hl=en&sa=X&ved=0ahUKEwjJhcjFn6rQAhUKkJQKHbw6AiwQ6AEIMTAE#v=onepage&q=Tryptophan%20LD-50&f=false
                                                #http://www.livestrong.com/article/541961-how-much-tryptophan-per-day/
        17 <= intake("Linoleic acid (g)") 

        45 <= intake("Selenium (Se) (µg)") <= 400 #https://en.wikipedia.org/wiki/Dietary_Reference_Intake
        11 <= intake("Zinc (Zn) (mg)") <= 40 #https://en.wikipedia.org/wiki/Dietary_Reference_Intake
            
            
        6 <= intake("Vitamin E (mg)") <= 1_200    #https://www.livestrong.com/article/465706-vitamin-e-overdose-levels/
            
        1.3 <= intake("Vitamin B6 (mg)") <= 300  #https://www.news-medical.net/health/Can-You-Take-Too-Much-Vitamin-B.aspx
        2.4 <= intake("Vitamin B12  (µg)")   #https://www.news-medical.net/health/Can-You-Take-Too-Much-Vitamin-B.aspx
            
    end)
    
end



In [ ]:
filter(x->occursin(x, "Vitamin B"), String.(collect(names(nutrients))))

### Function below is to show diet example

In [ ]:
function show_diet(diet)
    for ii in eachindex(diet)
        amount = getvalue(diet[ii])
        if amount > 0
            name = nutrients[ii, Symbol("Food Name")]
            grams = round(Int, 100amount) # amount is expressed in units of 100 grams
            display_as = grams < 1  ? ">0" : string(grams) #Make things that are "0.001 grams" show as ">0 grams"
            println(lpad(display_as, 5, " "), " grams \t", name)
        end
    end
end
`

# 4. Create first example - diet with minimal needs for 62 kg person
## Liquids are less than 25% of total body mass
### Informations are taken from https://en.wikipedia.org/wiki/Dietary_Reference_Intake

In [ ]:
m, diet = new_diet()
basic_nutrient_requirements(m, diet);

@constraint(m, diet .<= 5) # nothing more than 500g
@constraint(m, total_intake("Moisture (g)", diet) <= 0.25sum(diet)*100) # Less than 25% liquid

@objective(m, Min,  
    total_intake("Energy, with dietary fibre (kJ)", diet) + 0.01sum(diet)
)
@show status = solve(m)
show_diet(diet)



# 5. Create second example - diet for endurance athlete weighing 70 kg
## Maximizing protein intake is included
### based on articles from https://www.nal.usda.gov/fnic/dri-tables-and-application-reports
### recommendation for man 19-30 years old

In [ ]:
function endurance_athlete_requirements(m::Model, diet, weight = 70) # his weight is 70 kg
    intake(nutrient_name) = total_intake(nutrient_name, diet) 
    # ^ closure over the diet given as an argument
        
    @constraints(m, begin
        91 <= intake("Protein (g)") 
        70 <= intake("Total fat (g)") 
        24 <= intake("Total saturated fat (g)") 
        70 <= intake("Total sugars (g)") 
        intake("Alcohol (g)") <= 0
        350 <= intake("Available carbohydrates, with sugar alcohols (g)") 
        intake("Sodium (Na) (mg)") <= 2300 # Heart Council
        intake("Caffeine (mg)") <= 400
        25 <=intake("Dietary fibre (g)") 
        24 <= intake("Total trans fatty acids (mg)") 
        900 <= intake("Vitamin A retinol equivalents (µg)") <= 3000
        1.2 <= intake("Thiamin (B1) (mg)") 
        400 <= intake("Dietary folate equivalents  (µg)") 
        1000 <= intake("Folic acid  (µg)") # https://ods.od.nih.gov/factsheets/Folate-HealthProfessional/
        75 <= intake("Vitamin C (mg)") <= 2000 ##https://en.wikipedia.org/wiki/Dietary_Reference_Intake
        #600 <= intake("Vitamin D (IU)") <= 4000
        95 <= intake("Iodine (I) (µg)") <=1100 ##https://en.wikipedia.org/wiki/Dietary_Reference_Intake
        6 <= intake("Iron (Fe) (mg)") <= 45 # #https://en.wikipedia.org/wiki/Dietary_Reference_Intake
        330 <= intake("Magnesium (Mg) (mg)") 
        580 <= intake("Phosphorus (P) (mg)") <= 4000 #https://en.wikipedia.org/wiki/Dietary_Reference_Intake
        4700 <= intake("Potassium (K) (mg)") 
        800 <= intake("Calcium (Ca) (mg)") <= 2500 #https://en.wikipedia.org/wiki/Dietary_Reference_Intake
        3.5*weight <= intake("Tryptophan (mg)") <= 0.5*weight*1600 #0.5 * LD50 https://books.google.com.au/books?id=he7LBQAAQBAJ&pg=PA210&lpg=PA210&dq=Tryptophan+LD-50&source=bl&ots=yzi5Hx7hM2&sig=sd78LGAnavIwtT5AP-L4fzsFLZM&hl=en&sa=X&ved=0ahUKEwjJhcjFn6rQAhUKkJQKHbw6AiwQ6AEIMTAE#v=onepage&q=Tryptophan%20LD-50&f=false
                                                #http://www.livestrong.com/article/541961-how-much-tryptophan-per-day/
        17 <= intake("Linoleic acid (g)")
        #550 <= intake("Choline (g)")
        #16 <= intake("Niacin (g)")
        #5 <= intake("Patothenic Acid (mg)")
        
        45 <= intake("Selenium (Se) (µg)") <= 400 #https://en.wikipedia.org/wiki/Dietary_Reference_Intake
        11 <= intake("Zinc (Zn) (mg)") <= 40 #https://en.wikipedia.org/wiki/Dietary_Reference_Intake
            
            
        15 <= intake("Vitamin E (mg)") <= 1000    #https://www.livestrong.com/article/465706-vitamin-e-overdose-levels/
        #120 <= intake("Vitamin K (µg)") 
            
        1.3 <= intake("Vitamin B6 (mg)") <= 100  #https://www.news-medical.net/health/Can-You-Take-Too-Much-Vitamin-B.aspx
        2.4 <= intake("Vitamin B12  (µg)")   #https://www.news-medical.net/health/Can-You-Take-Too-Much-Vitamin-B.aspx
        #1.3 <= intake("Vitamin B2 (mg)")
        
    end)
    
end

In [ ]:
m, diet = new_diet()
endurance_athlete_requirements(m, diet);
forbid_liquids(m, diet)
@constraint(m, diet .<= 3) # nothing more than 300g


@constraint(m,
    9_000 <=
        total_intake("Energy, with dietary fibre (kJ)", diet)
    <= 12_000
    )
@objective(m, Max,  
    total_intake("Protein (g)", diet)
)
@show status = solve(m)
show_diet(diet)

# 6. Create third example - diet for 20 travelling people
## Included minimal weight of food
### Function is exactly the same as in example 1

In [ ]:
function total_intake(nutrient_name, diet)
    nutrients[:, Symbol(nutrient_name)] ⋅ diet
end

function basic_nutrient_group(m::Model, diet, weight = 62) # average human is 62 kg
    intake(nutrient_name) = total_intake(nutrient_name, diet) 
    # ^ closure over the diet given as an argument
        
    @constraints(m, begin
        70 <= intake("Protein (g)") 
        70 <= intake("Total fat (g)") 
        24 <= intake("Total saturated fat (g)") 
        90 <= intake("Total sugars (g)") 
        intake("Alcohol (g)") <= 100
        310 <= intake("Available carbohydrates, with sugar alcohols (g)") 
        intake("Sodium (Na) (mg)") <= 2300 # Heart Council
        intake("Caffeine (mg)") <= 400
        25 <=intake("Dietary fibre (g)") 
        24 <= intake("Total trans fatty acids (mg)") 
        25 <= intake("Vitamin A retinol equivalents (µg)") <= 900
        1.2 <= intake("Thiamin (B1) (mg)") 
        400 <= intake("Dietary folate equivalents  (µg)") 
        1000 <= intake("Folic acid  (µg)") # https://ods.od.nih.gov/factsheets/Folate-HealthProfessional/
        75 <= intake("Vitamin C (mg)") <= 2000 ##https://en.wikipedia.org/wiki/Dietary_Reference_Intake
        95 <= intake("Iodine (I) (µg)") <=1100 ##https://en.wikipedia.org/wiki/Dietary_Reference_Intake
        6 <= intake("Iron (Fe) (mg)") <= 45 # #https://en.wikipedia.org/wiki/Dietary_Reference_Intake
        330 <= intake("Magnesium (Mg) (mg)") 
        580 <= intake("Phosphorus (P) (mg)") <= 4000 #https://en.wikipedia.org/wiki/Dietary_Reference_Intake
        4700 <= intake("Potassium (K) (mg)") 
        800 <= intake("Calcium (Ca) (mg)") <= 2500 #https://en.wikipedia.org/wiki/Dietary_Reference_Intake
        3.5*weight <= intake("Tryptophan (mg)") <= 0.5*weight*1600 #0.5 * LD50 https://books.google.com.au/books?id=he7LBQAAQBAJ&pg=PA210&lpg=PA210&dq=Tryptophan+LD-50&source=bl&ots=yzi5Hx7hM2&sig=sd78LGAnavIwtT5AP-L4fzsFLZM&hl=en&sa=X&ved=0ahUKEwjJhcjFn6rQAhUKkJQKHbw6AiwQ6AEIMTAE#v=onepage&q=Tryptophan%20LD-50&f=false
                                                #http://www.livestrong.com/article/541961-how-much-tryptophan-per-day/
        17 <= intake("Linoleic acid (g)") 

        45 <= intake("Selenium (Se) (µg)") <= 400 #https://en.wikipedia.org/wiki/Dietary_Reference_Intake
        11 <= intake("Zinc (Zn) (mg)") <= 40 #https://en.wikipedia.org/wiki/Dietary_Reference_Intake
            
            
        6 <= intake("Vitamin E (mg)") <= 1_200    #https://www.livestrong.com/article/465706-vitamin-e-overdose-levels/
            
        1.3 <= intake("Vitamin B6 (mg)") <= 300  #https://www.news-medical.net/health/Can-You-Take-Too-Much-Vitamin-B.aspx
        2.4 <= intake("Vitamin B12  (µg)")   #https://www.news-medical.net/health/Can-You-Take-Too-Much-Vitamin-B.aspx
            
    end)
    
end

In [ ]:
m, diet = new_diet()
basic_nutrient_group(m, diet);
forbid_liquids(m, diet)

@constraint(m, diet .<= 5) # nothing more than 500g
@constraint(m, total_intake("Moisture (g)", diet) <= 0.25sum(diet)*100) # Less than 25% liquid

# @objective(m, Min,  
#      total_intake("Energy, with dietary fibre (kJ)", diet) + 0.01sum(diet)
#  )
@objective(m, Min, sum(diet))
@show status = solve(m)
show_diet(diet*20)

println("Total: $(round(Int, 100sum(getvalue(diet)))) grams")

# References:
### https://en.wikipedia.org/wiki/Dietary_Reference_Intake
### https://white.ucc.asn.au/2018/05/28/Optimizing-your-diet-with-JuMP.html
### https://neos-guide.org/content/diet-problem
### https://www.nal.usda.gov/fnic/dri-tables-and-application-reports
### https://github.com/JuliaOpt/JuMP.jl